#Дисперсионный анализ ANOVA
Загрузим данные в сессию:

In [52]:
import numpy as np
import scipy as sp

cities = open("cities.txt")
citiestemp = open("cities.txt.tmp", "w+")
for line in cities:
    citiestemp.write(line.replace(",", "."))
cities.close()
citiestemp.close()
data = np.genfromtxt("cities.txt.tmp", names=True, usecols=("CITY", "STATE", "CRIME", "INCOME", "UNEMP"), dtype=None)

Зададимся задачей проанализировать разницу в доходах в разных штатах. Тогда нам понадобятся колонки `"STATE"` и `"INCOME"`, причём `"STATE"` будет определять подпопуляцию. Предположим, что наблюдения подчиняются модели
$$x_{ij} = \mu + a_i + \varepsilon_{ij}.$$
Оценим тогда параметры этой модели:

In [59]:
states = set(data["STATE"])
mu = np.mean(data["INCOME"])

77 34
